## Will it Fit?  
**A container problem solved.**

The containerstore.com has a large selection of containers, but does not provide a search option to specify the size of the container.  My cat clawed the wicker baskets that fit perfectly in my small dresser, so I am in need of replacement containers in a very specific size.  Then I will be able to avoid trashing the entire piece of furniture.

First I scraped the data from the container store website collecting the name, dimensions, and url for each piece.  In this notebook we will be able to search for containers within a given range of appropriate dimensions.


In [1]:
import scrapy
import json
from pprint import pprint

Let's start by looking at the collected data.  

In [2]:
with open('container.json') as data_file:    
    data = json.load(data_file)

# pprint(data[0:10])
# count = 0
for i in range(0,len(data)):

    if((data[i]['dimensions'])==[]):
        print('no dimensions given:')
        print (data[i]['url'])
#         count += 1
pprint(len(data))
# print(count)

no dimensions given:
https://www.containerstore.com/s/storage/decorative-bins-baskets/folding-wire-storage-basket/12d?productId=11000564
203


We scraped 203 items stored as json from the "decorative bins and baskets" section of the container store website.  There is only one container without dimensions given.

In [3]:
for i in range (0,10):
    print(data[i]['dimensions'][0])

10" sq. x 10" h
13-1/4" x 9-1/2" x 5" h
16" x 11" x 9" h
9-1/4" x 14-1/4" x 4-1/2" h
13" x 18" x 9" h
12-1/4" x 8-1/2" x 7" h
9-1/2" x 16-1/2" x 9-1/2" h
12-1/4" x 8-1/2" x 7" h
8-1/2" x 13" x 9" h
11" x 15" x 6-1/4" h


Next we adjust the format of the dimensions to make them searchable. They will be listed as [length, width, height] all in inches. 

In [4]:
from fractions import Fraction

for j in range (0,len(data)):
    
    data[j]['new dimensions']=[]
    
    # list of three dimensions (or two if len=width, height always last)
    
    for k in range(0,len(data[j]['dimensions'])):
        mydim = data[j]['dimensions'][k].split('x')

        newdim = []

        # if the len = width it says "sq." and only has two entries, first the len/width, second the height
        if len(mydim)==2:
            for i in range (0,len(mydim)):
                spot = mydim[i]
                squaresplit = spot.split('sq.')[0]
                mysplit = squaresplit.split('-')
                if len(mysplit)==2:

                    integer = mysplit[0]
                    fraction = float(Fraction(mysplit[1].split('"')[0]))
                    num = float(integer) + fraction
                else:
                    num = float(mysplit[0].split('"')[0])
                if (i==0):
                    newdim.append(num)

                newdim.append(num)

        else:
            for i in range (0,len(mydim)):

                spot = mydim[i]
                mysplit = spot.split('-')
                if len(mysplit)==2:

                    integer = mysplit[0]
                    fraction = float(Fraction(mysplit[1].split('"')[0]))
                    num = float(integer) + fraction
                else:
                    num = float(mysplit[0].split('"')[0])

                newdim.append(num)

        data[j]['new dimensions'].append(newdim)

In [5]:
for i in range (0,10):
    print(data[i]['new dimensions'][0])

[10.0, 10.0, 10.0]
[13.25, 9.5, 5.0]
[16.0, 11.0, 9.0]
[9.25, 14.25, 4.5]
[13.0, 18.0, 9.0]
[12.25, 8.5, 7.0]
[9.5, 16.5, 9.5]
[12.25, 8.5, 7.0]
[8.5, 13.0, 9.0]
[11.0, 15.0, 6.25]


In [6]:
print(data[0]['dimensions'])
print(data[0]['new dimensions'])

['10" sq. x 10" h', '12" sq. x 12" h']
[[10.0, 10.0, 10.0], [12.0, 12.0, 12.0]]


The will_it_fit function will be given 6 values, 2 for each dimension (a lower and upper limit).  It will return the possible containers.

In [7]:
def will_it_fit(f1a,f1b,f2a,f2b,f3a,f3b):

    possibles = []
    for j in range(0,len(data)):

        k=0
        if (len(data[j]['new dimensions'])>0):
            if ((f1a<data[j]['new dimensions'][k][0]<f1b) and (f2a<data[j]['new dimensions'][k][1]<f2b) 
                and (f3a<data[j]['new dimensions'][k][2]<f3b)):
                
                possibles.append([data[j],k])
            
    return possibles
    

The container that I am looking for should be in the range of 6-8.25" x 9-12" x 5-7"

In [8]:
f1a = 6.
f1b = 8.25
f2a = 9.
f2b = 12.
f3a = 5.
f3b = 7.


possibles = will_it_fit(f1a,f1b,f2a,f2b,f3a,f3b)
for c in possibles:
    print('')
    print(c[0]['title'],c[0]['dimensions'])
    print(c[0]['url'])



Rectangular Hogla Storage Bin with Lid ['7" x 11" x 5-1/4" h']
https://www.containerstore.com/s/storage/decorative-bins-baskets/rectangular-hogla-storage-bin-with-lid/12d?productId=11003202

Mondrian Storage Boxes with Lids ['7" x 10-5/8" x 5-3/8" h', '10-3/4" x 14-5/8" x 5-7/16" h', '10-3/4" x 14-5/8" x 9-5/16" h']
https://www.containerstore.com/s/storage/decorative-bins-baskets/mondrian-storage-boxes-with-lids/12d?productId=10036979

Small Rectangular Hogla Storage Bin with Handles ['7" x 11" x 5-1/4" h']
https://www.containerstore.com/s/storage/decorative-bins-baskets/small-rectangular-hogla-storage-bin-with-handles/12d?productId=10035370
